# Snapshot from .vtp

In [3]:
import pathlib
import warnings

from cfdmod.use_cases.snapshot.camera import take_snapshot
from cfdmod.use_cases.snapshot.config import LegendConfig, ImageConfig, PartialProjectionConfig, SnapshotConfig
from cfdmod.use_cases.snapshot.process_image import process_image
from cfdmod.utils import create_folder_path

<div style="text-align: center"> edit: </div>

- define the paths

In [19]:
files_path = pathlib.Path("./fixtures/tests/snapshot")
output_path = pathlib.Path("./output/images")
snapshot_config_path = files_path / "snapshot_params_example.yaml"
create_folder_path(output_path)

- wind_dirs: a list of strings that represents wind directions, used to create the path to each projection in each image
- images: configuration to generate images
  - name: name that represents the image
  - legend_config: configuration of the label in the image
  - projections: a set of key value pair of projections names and partial projection config (only the part thats differs in each projection)

In [8]:
wind_dirs = ["006", "036"]
images = [
    ImageConfig(
        name= f"{wind_dir}",
        legend_config = LegendConfig(label="label", range=[-100, 100], n_divs=10),
        projections={
                "top_projection": PartialProjectionConfig(file_path=files_path / f"loads_cupula_top_{wind_dir}.vtp", scalar="max [kgf/m²]"),
                "left_projection": PartialProjectionConfig(file_path=files_path / f"loads_cupula_top_{wind_dir}.vtp", scalar="min [kgf/m²]"),
                "right_projection": PartialProjectionConfig(file_path=files_path / f"loads_cupula_top_{wind_dir}.vtp", scalar="min [kgf/m²]"),
                "bottom_projection": PartialProjectionConfig(file_path=files_path / f"loads_cupula_bottom_{wind_dir}.vtp", scalar="max [kgf/m²]"),
            },
        )
    for wind_dir in wind_dirs
]

<div style="text-align: center"> execute: </div>

- a base snapshot configuration is created using an YAML file. Each image configuration subscribes some parts, creating the final configuration for the image.
- For each image configuration, the snapshot base configuration will be modified according to the settings for the respective image.

In [ ]:
snapshot_config = SnapshotConfig.from_file(snapshot_config_path)

for image in images:
    for projection_key in image.projections:
        snapshot_config.name = output_path / image.name
        snapshot_config.projections[projection_key].file_path = image.projections[projection_key].file_path
        snapshot_config.projections[projection_key].scalar = image.projections[projection_key].scalar
        snapshot_config.legend_config.label = image.legend_config.label
        snapshot_config.legend_config.range = image.legend_config.range
        snapshot_config.legend_config.n_divs = image.legend_config.n_divs
        
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        take_snapshot(snapshot_config)
    process_image(
        image_path=output_path / f"{image.name}.png",
        output_path=output_path / f"{image.name}.png",
        crop_cfg=snapshot_config.camera.crop,
    )